In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 500)

import warnings
import gc
import time
from tqdm import tqdm_notebook,tnrange


#import urllib
#import sys
#import urllib.request
from urllib.request import urlopen
from urllib.parse import quote
import json
import requests

In [2]:
def merge_data():
    path = './'
    train_plans_I   = pd.read_csv(path + 'train_plans_phase1.csv', parse_dates=['plan_time'])
    train_plans_II   = pd.read_csv(path + 'train_plans_phase2.csv', parse_dates=['plan_time'])
    train_queries_I = pd.read_csv(path + 'train_queries_phase1.csv', parse_dates=['req_time'])
    train_queries_II = pd.read_csv(path + 'train_queries_phase2.csv', parse_dates=['req_time'])
    test_plans    = pd.read_csv(path + 'test_plans.csv', parse_dates=['plan_time'])
    test_queries  = pd.read_csv(path + 'test_queries.csv', parse_dates=['req_time'])
    train_clicks_I  = pd.read_csv(path + 'train_clicks_phase1.csv')
    train_clicks_II  = pd.read_csv(path + 'train_clicks_phase2.csv')
    profiles      = pd.read_csv(path + 'profiles.csv') 
    
    train_I = train_queries_I.merge(train_plans_I, 'left', ['sid'])
    train_II = train_queries_II.merge(train_plans_II, 'left', ['sid'])
    test  = test_queries.merge(test_plans, 'left', ['sid'])
    train_I = train_I.merge(train_clicks_I, 'left', ['sid'])
    train_II = train_II.merge(train_clicks_II, 'left', ['sid'])
    train_I['click_mode'] = train_I['click_mode'].fillna(0).astype(int)
    train_II['click_mode'] = train_II['click_mode'].fillna(0).astype(int)
    data  = pd.concat([train_I, train_II, test], ignore_index=True)
    data  = data.merge(profiles, 'left', ['pid']) 

    return data

In [15]:
data =merge_data()

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



#### POI
地铁, 公交, 医院, 大学, 中学, 住宅, 写字楼, 美食, 娱乐, 金融, 旅游, 火车站, 机场

In [23]:
#trial
json.load(urlopen('http://api.map.baidu.com/place/v2/search?query=%E5%85%AC%E4%BA%A4%E7%AB%99&bounds=39.7749,115.4349,39.7851,115.4451&output=json&page_size=20&coord_type=1&ak=LlNVsWOsqmKwlqkUEBmzbpmbu2GAc2Dl'))

{'status': 0,
 'message': 'ok',
 'total': 1,
 'results': [{'name': '鱼斗泉',
   'location': {'lat': 39.782902, 'lng': 115.452537},
   'address': '房19路',
   'province': '北京市',
   'city': '北京市',
   'area': '房山区',
   'detail': 0,
   'uid': '24cdb2fa9182075636b06c82'}]}

In [16]:
data.drop_duplicates(subset=['o'],inplace=True)

In [17]:
data.shape

(15203, 75)

In [18]:
data['o_lng'] = data['o'].apply(lambda x: float(x.split(',')[0]))
data['o_lat'] = data['o'].apply(lambda x: float(x.split(',')[1]))
data['d_lng'] = data['d'].apply(lambda x: float(x.split(',')[0]))
data['d_lat'] = data['d'].apply(lambda x: float(x.split(',')[1])) 

In [19]:
data = data.reset_index(drop=True)

In [20]:
res = pd.DataFrame()
res['o'] = data['o']

In [43]:
res.head()

,o,district_o
0,"116.34,39.99",海淀区
1,"116.43,39.94",东城区
2,"116.28,40.01",海淀区
3,"116.31,39.51",大兴区
4,"116.33,39.89",西城区


In [29]:
url = 'http://api.map.baidu.com/place/v2/search?query=%E5%85%AC%E4%BA%A4%E7%AB%99'+f'&bounds={lat_0},{lng_0},{lat_1},{lng_1}&output=json&page_size=20&coord_type=1&ak=LlNVsWOsqmKwlqkUEBmzbpmbu2GAc2Dl'
response = urlopen(url)
page = json.load(response)
page.area

{'status': 0,
 'message': 'ok',
 'total': 1,
 'results': [{'name': '尚信村西',
   'location': {'lat': 40.143219, 'lng': 116.387493},
   'address': '昌52路',
   'province': '北京市',
   'city': '北京市',
   'area': '昌平区',
   'detail': 0,
   'uid': 'f003b73ec5873b38f4b91683'}]}

In [48]:
lat_1

39.9051

In [51]:
#coor_df - dataframe of unique coordinates
res['district_o'] = ''
tube_stops = []

#unique_coor = coor_df['coordinates'].tolist()
for i in tqdm_notebook(range(len(data))):
    lat = data['o_lat'][i]
    lng = data['o_lng'][i]
    
    lat_0 = round(lat - 0.51, 4)
    lng_0 = round(lng - 0.51, 4)
    lat_1 = round(lat + 0.51, 4)
    lng_1 = round(lng + 0.51, 4)
    
    #url = 'http://api.map.baidu.com/place/v2/search?query=' + quote('停车场') + f'&bounds={lat_0},{lng_0},{lat_1},{lng_1}&output=json&page_size=20&coord_type=1&ak=LlNVsWOsqmKwlqkUEBmzbpmbu2GAc2Dl'
    url = 'http://api.map.baidu.com/place/v2/search?query=%E5%85%AC%E4%BA%A4%E7%AB%99'+f'&bounds={lat_0},{lng_0},{lat_1},{lng_1}&output=json&page_size=20&coord_type=1&ak=LlNVsWOsqmKwlqkUEBmzbpmbu2GAc2Dl'
    response = urlopen(url)
    page = json.load(response)  
    tube_stops.append(page)
    res.at[i, 'district_o'] = str(page['results'][0]['area'])


KeyError: 'area'

In [14]:

res.to_csv('./extra_I.csv',index=False)

In [52]:
res.drop(['bus_stops_o'],inplace=True,axis=1)

In [11]:
test_queries['o_lng_trans'] = 0
test_queries['o_lat_trans'] = 0

In [12]:
# 坐标转换爬虫
for i in tqdm(range(len(test_queries))):
    lat = test_queries['o_lat'][i]
    lng = test_queries['o_lng'][i]
    
    url_bd = f'http://api.map.baidu.com/geoconv/v1/?coords={lng},{lat}&from=1&to=5&ak=QO908zX60IUFFEFlw5lIw7vOnQcabYFH'
    response_bd = urlopen(url_bd)
    
    result = json.load(response_bd)['result'][0]
    
    #print (result['x'])
    test_queries['o_lng_trans'][i] = round(result['x'], 6)
    test_queries['o_lat_trans'][i] = round(result['y'], 6)
    
    time.sleep(0.1)


  0%|                                                                                       | 0/304916 [00:00<?, ?it/s]E:\Anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]

  0%|                                                                           | 23/304916 [00:05<18:56:05,  4.47it/s]

KeyboardInterrupt: 

In [11]:
json.load(response_bd)

{'status': 220, 'message': 'APP Referer校验失败'}